In [1]:
import os
import sys
import lxml
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Polygon
from shapely.geometry import Point


from bs4 import BeautifulSoup

/home/renanop/.pyenv/versions/py311_tcc/lib/python3.11/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.8.0-CAPI-1.13.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
# KML file
basedir = os.getcwd()
filename = 'data/doc.kml'

path = os.path.join(basedir, filename)

# Bike data
rel_path_bike_data = 'data/Dados Brutos UFRJ Out22.xlsx'
bike_data_path = os.path.join(basedir, rel_path_bike_data)

rel_path_coords = 'data/MapaCalorOut22.xlsx'
coords_path = os.path.join(basedir, rel_path_coords)



## 1 - Processing KML data to extract location coordinates.

In [3]:
with open(path, 'r',encoding="utf-8") as kml_file:
    soup = BeautifulSoup(kml_file, features = 'xml')
    

In [4]:
names  = soup.findAll('name')
# Removing bad tags
names.pop(0) # Removing bad tags
names.pop(0) # Removing bad tags
coordinates = soup.findAll('coordinates')

print(names)

[<name>CT2</name>, <name>CT</name>, <name>NQRT</name>, <name>IEN</name>, <name>ZONA INDUSTRIAL/INCUBADORA</name>, <name>PARQUE TECNOLÓGICO</name>, <name>CETEM</name>, <name>COPEAD</name>, <name>VILA RESIDENCIAL</name>, <name>REITORIA</name>, <name>LETRAS</name>, <name>CCMN</name>, <name>LADETEC</name>, <name>CENPES</name>, <name>Polígono 15</name>, <name>RU CENTRAL</name>, <name>EEFD</name>, <name>CCS</name>, <name>ESTAÇÃO DE INTEGRAÇÃO</name>, <name>HU/IPPMG</name>, <name>BIO RIO</name>, <name>ALOJAMENTO</name>, <name>HANGAR</name>, <name>OFICÍNA</name>, <name>SERTTEL FILIAL</name>, <name>CPMR</name>]


In [5]:
def clean_coords(elem):
    
    # get text from coordinates tag
    text = elem.get_text()
    
    # clean weird characters from text
    text_lst = text.replace(' ', '').replace(',0','').split('\n')
    
    # removing empty strings
    text_lst_treated = [i for i in text_lst if len(i) > 0]
    
    vertex_dic = {}
    for ind,v in enumerate(text_lst_treated):
        key = f'vertex_{ind}'
        lst = v.split(',')
        lst_float = [float(k) for k in lst]
        vertex_dic.update({key:lst_float})
    
    return vertex_dic

In [6]:
# Creating locations dictionary and iterating through locations with bs4.
locations = {}
for name, coord in zip(names, coordinates):
    new_kv = {name.get_text():clean_coords(coord)}
    locations.update(new_kv)

In [7]:
def clean_coords_v2(elem):
    
    # get text from coordinates tag
    text = elem.get_text()
    
    # clean weird characters from text
    text_lst = text.replace(' ', '').replace(',0','').split('\n')
    
    # removing empty strings
    text_lst_treated = [i for i in text_lst if len(i) > 0]
    
    latitudes = []
    longitudes = []
    
    for pair in text_lst_treated:
        lat = float(pair.split(',')[0])
        long = float(pair.split(',')[1])

        latitudes.append(lat)
        longitudes.append(long)
    
    
    coords_dict = {'latitudes': latitudes , 'longitudes':longitudes}
    
    return coords_dict
    


In [8]:
# Creating locations dictionary and iterating through locations with bs4.
locations = {}
for name, coord in zip(names, coordinates):
    new_kv = {name.get_text():clean_coords_v2(coord)}
    locations.update(new_kv)

In [9]:
# Example of output
locations

{'CT2': {'latitudes': [-43.231289,
   -43.230908,
   -43.229739,
   -43.22961,
   -43.230458,
   -43.231477,
   -43.230769,
   -43.231289],
  'longitudes': [-22.865004,
   -22.865187,
   -22.864495,
   -22.86404,
   -22.862973,
   -22.863412,
   -22.864391,
   -22.865004]},
 'CT': {'latitudes': [-43.22954,
   -43.229122,
   -43.226558,
   -43.22689,
   -43.2272922,
   -43.2277033,
   -43.2279204,
   -43.230436,
   -43.23072,
   -43.231772,
   -43.231917,
   -43.231032,
   -43.231611,
   -43.229824,
   -43.229878,
   -43.22998,
   -43.22954],
  'longitudes': [-22.863838,
   -22.864362,
   -22.862771,
   -22.862296,
   -22.8624324,
   -22.8618437,
   -22.8615569,
   -22.858505,
   -22.858114,
   -22.858797,
   -22.859098,
   -22.86025,
   -22.860616,
   -22.862766,
   -22.863043,
   -22.863275,
   -22.863838]},
 'NQRT': {'latitudes': [-43.228976,
   -43.229356,
   -43.229013,
   -43.228632,
   -43.228976],
  'longitudes': [-22.864928, -22.865254, -22.865625, -22.865388, -22.864928]},
 'I

## 2 - Creating DataFrame of Polygons (Virtual Stations)

In [10]:
def create_locations_df(locations_dict):
    """A function that creates a GeoDataframe of location names as indexes, and location polygons as the geometry"""
    
    polygon_list = []
    for enum, locs in enumerate(locations.keys()):
        lat_point_list = locations[locs]['latitudes']
        lon_point_list = locations[locs]['longitudes']

        polygon_geom = Polygon(zip(lat_point_list, lon_point_list))
        polygon = gpd.GeoDataFrame(index=[locs], crs='epsg:4326', geometry=[polygon_geom])

        polygon_list.append(polygon)



    all_polygons = pd.concat(polygon_list)
    
    return all_polygons
    
all_polygons = create_locations_df(locations)

In [11]:
# Example of output: All locations (indexes) as shapely polygons in a GeoDataFrame
all_polygons

,geometry
CT2,"POLYGON ((-43.23129 -22.86500, -43.23091 -22.8..."
CT,"POLYGON ((-43.22954 -22.86384, -43.22912 -22.8..."
NQRT,"POLYGON ((-43.22898 -22.86493, -43.22936 -22.8..."
IEN,"POLYGON ((-43.22752 -22.86546, -43.22704 -22.8..."
ZONA INDUSTRIAL/INCUBADORA,"POLYGON ((-43.22086 -22.86567, -43.21993 -22.8..."
PARQUE TECNOLÓGICO,"POLYGON ((-43.21915 -22.86489, -43.21833 -22.8..."
CETEM,"POLYGON ((-43.22220 -22.86049, -43.22081 -22.8..."
COPEAD,"POLYGON ((-43.21833 -22.85965, -43.21893 -22.8..."
VILA RESIDENCIAL,"POLYGON ((-43.21813 -22.86728, -43.21811 -22.8..."
REITORIA,"POLYGON ((-43.22476 -22.86216, -43.22359 -22.8..."


In [12]:
# Plotting Locations
m = folium.Map([-22.86546 ,-43.22752], zoom_start=14, tiles='cartodbpositron')
folium.GeoJson(all_polygons).add_to(m)
folium.LatLngPopup().add_to(m)

In [13]:
display(m)

locations.keys()

## 3 - Processing bike operations data

In [14]:
# Importing DataFrames
def read_raw_bike_data(path):
    
    # Removing useless columns
    df = pd.read_excel(bike_data_path).drop(['EstacaoRetirada',
                                                       'AreaEstacaoRetirada', 
                                                       'EnderecoEstacaoRetirada', 
                                                       'EstacaoDevolucao',
                                                       'AreaEstacaoDevolucao',
                                                       'EnderecoEstacaoDevolucao',
                                                       'Nome',
                                                       'Email',
                                                       'Celular',
                                                       'Projeto'
                                            ], axis = 1)
    
    
    # Unpacking time information
    df['hora_retirada'] =  df['HoraRetirada'].astype(str).str.split(':').str[0]
    df['minuto_retirada'] =  df['HoraRetirada'].astype(str).str.split(':').str[1]
    df['segundo_retirada'] =  df['HoraRetirada'].astype(str).str.split(':').str[2]
    
    df['hora_devolucao'] =  df['HoraDevolucao'].astype(str).str.split(':').str[0]
    df['minuto_devolucao'] =  df['HoraDevolucao'].astype(str).str.split(':').str[1]
    df['segundo_devolucao'] =  df['HoraDevolucao'].astype(str).str.split(':').str[2]
    
    # Creating adequate timestamps
    df['duracao_corrida(min)'] = pd.to_timedelta(df['[Duração da Corrida]'].str.split(' ').str[0].astype(int), 'minute')
    df['removal_timestamp'] = pd.to_datetime(df['DataCorrida'].astype(str) + ' ' +  df['HoraRetirada'].astype(str))
    df['return_timestamp'] = df['removal_timestamp'] + df['duracao_corrida(min)']
    
    
    
    return df

# Function Calls
raw_bike_data_df = read_raw_bike_data(bike_data_path)
bike_coords_df_removal  = pd.read_excel(coords_path, sheet_name = 'Retirada')
bike_coords_df_return  = pd.read_excel(coords_path, sheet_name = 'Devolucao')


In [15]:
def create_trip_database(bike_raw_data, removals, returns, removal_or_return):
    """A function that creates the trip database given the raw data and removals and returns location data"""
    # Joining base with removal and return coordinates
    trip_database = (bike_raw_data
                      .merge(removals.add_suffix('_removal'),how = 'inner', left_on = 'IdJornada', right_on = 'Jornada_removal')
                      .merge(returns.add_suffix('_return'), how = 'inner', left_on = 'IdJornada', right_on = 'Jornada_return'))
    
    assert removal_or_return in ('removal', 'return')
    
    if removal_or_return == 'removal':

        trip_database = gpd.GeoDataFrame(
        trip_database, 
        geometry = gpd.points_from_xy(trip_database['Longitude_removal'], trip_database['Latitude_removal'], crs = 'EPSG:4326')
    )
        
    elif removal_or_return == 'return':
    
        trip_database = gpd.GeoDataFrame(
        trip_database, 
        geometry = gpd.points_from_xy(trip_database['Longitude_return'], trip_database['Latitude_return'], crs = 'EPSG:4326')
    )
        
    # Transforming to crs 3857, which is adequate for snearest spatial join
    trip_database = trip_database.to_crs(crs = 3857)
    
    return trip_database

# Function Calls
trips_removal_base = create_trip_database(raw_bike_data_df, bike_coords_df_removal, bike_coords_df_return, 'removal')
trips_return_base = create_trip_database(raw_bike_data_df, bike_coords_df_removal, bike_coords_df_return, 'return')

In [16]:
def create_full_database_with_final_destinations(removal_base, return_base, polygons):
    """A function that performs the spatial nearest join"""
    removal_df = removal_base.sjoin_nearest(polygons.to_crs(crs = 3857), how = 'left', distance_col = 'Distances')
    return_df = return_base.sjoin_nearest(polygons.to_crs(crs = 3857), how = 'left', distance_col = 'Distances')
    
    join_indexes = [i for i in list(removal_df) if i not in ('geometry', 'index_right', 'Distances')]
    
    full_base = removal_df.merge(return_df, how = 'left', on = join_indexes, suffixes = ('_removal', '_return'))
    
    return full_base

# Function Calls
final_database = create_full_database_with_final_destinations(trips_removal_base, trips_return_base, all_polygons)
    

In [17]:
# Exporting
output = final_database.loc[:, ['IdJornada','Latitude_removal', 'Longitude_removal', 'Latitude_return', 'Longitude_return', 'index_right_removal', 'Distances_removal', 'index_right_return', 'Distances_return', 'duracao_corrida(min)', 'removal_timestamp', 'return_timestamp']]

output.to_csv('bike_trips_processed.csv')

In [18]:
final_database

,globalId,Sexo,Nascimento,País,Cidade,UF,[Data de Cadastro],IdJornada,DataCorrida,DiaSemana,...,Longitude_removal,Jornada_return,Latitude_return,Longitude_return,geometry_removal,index_right_removal,Distances_removal,geometry_return,index_right_return,Distances_return
0,1959805,M,1990-02-09,BR,Rio de Janeiro,RJ,2015-04-11 07:32:38.647,123383,2022-10-01,1900-01-07,...,-43.205431,123383,-22.902224,-43.205480,POINT (-4809606.591 -2620212.040),SERTTEL FILIAL,0.000000,POINT (-4809612.033 -2620198.613),SERTTEL FILIAL,0.000000
1,1959805,M,1990-02-09,BR,Rio de Janeiro,RJ,2015-04-11 07:32:38.647,123396,2022-10-01,1900-01-07,...,-43.205178,123396,-22.838907,-43.228769,POINT (-4809578.390 -2620205.595),SERTTEL FILIAL,0.000000,POINT (-4812204.540 -2612548.714),ALOJAMENTO,6.586758
2,6481360,-,2001-04-17,BR,-,-,2022-09-19 21:37:55.447,123501,2022-10-03,1900-01-02,...,-43.230600,123501,-22.860818,-43.226751,POINT (-4812408.379 -2614888.974),CT,0.000000,POINT (-4811979.922 -2615195.558),LETRAS,0.000000
3,6483248,-,2002-01-14,BR,-,-,2022-09-21 14:59:29.883,123502,2022-10-03,1900-01-02,...,-43.230764,123502,-22.860900,-43.226720,POINT (-4812426.685 -2614888.169),CT,0.000000,POINT (-4811976.459 -2615205.491),LETRAS,0.000000
4,6482795,-,1999-08-11,BR,-,-,2022-09-21 10:04:42.683,123503,2022-10-03,1900-01-02,...,-43.230556,123503,-22.860918,-43.226840,POINT (-4812403.431 -2614853.806),CT,23.739097,POINT (-4811989.817 -2615207.638),LETRAS,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7855,6499012,-,1999-08-31,BR,-,-,2022-10-05 16:13:23.367,133591,2022-10-31,1900-01-02,...,-43.217724,133591,-22.868602,-43.217249,POINT (-4810975.078 -2616046.074),VILA RESIDENCIAL,0.000000,POINT (-4810922.140 -2616136.014),VILA RESIDENCIAL,0.000000
7856,6496882,-,2003-03-17,BR,-,-,2022-10-04 11:48:57.557,133592,2022-10-31,1900-01-02,...,-43.217062,133592,-22.868567,-43.217373,POINT (-4810901.360 -2616117.758),VILA RESIDENCIAL,2.068282,POINT (-4810935.993 -2616131.719),VILA RESIDENCIAL,0.000000
7857,6510012,-,2002-09-16,BR,-,-,2022-10-14 10:03:53.990,133593,2022-10-31,1900-01-02,...,-43.217382,133593,-22.868498,-43.217147,POINT (-4810936.982 -2616143.532),VILA RESIDENCIAL,0.000000,POINT (-4810910.760 -2616123.396),VILA RESIDENCIAL,0.000000
7858,6502153,-,1999-11-20,BR,-,-,2022-10-08 01:08:03.363,133596,2022-10-31,1900-01-02,...,-43.217409,133596,-22.868702,-43.217320,POINT (-4810939.951 -2616125.812),VILA RESIDENCIAL,0.000000,POINT (-4810930.056 -2616148.096),VILA RESIDENCIAL,0.000000
